In [2]:
import pandas as pd
from google.colab import drive


drive.mount('/content/drive')
df_raw = pd.read_csv('/content/drive/MyDrive/FTC/train.csv')
df_raw.head()


Mounted at /content/drive


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [3]:
df1 = df_raw.copy()

In [4]:
# convering Delivery_person_Age
line_nan = (df1['Delivery_person_Age'] != 'NaN ') 
df1 = df1.loc[line_nan, :].copy()

df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)

# convering Delivery_person_Ratings
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype( float )

# converting Order_Date 
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format = '%d-%m-%Y')

# convertig multiple_deliveries
aux = (df1['multiple_deliveries']!= 'NaN ')
df1 = df1.loc[aux, :].copy()

df1['multiple_deliveries'] = df1['multiple_deliveries'].astype( int )

# removing spaces in strings or objects
#df1 = df1.reset_index(drop = True)
#for i in range(len(df1)):
#   df1.loc[i,'ID'] = df1.loc[i, 'ID'].strip()

df1.loc[:,'ID']=df1.loc[:,'ID'].str.strip()
df1.loc[:,'Delivery_person_ID']=df1.loc[:,'Delivery_person_ID'].str.strip()
df1.loc[:,'Type_of_order']=df1.loc[:,'Type_of_order'].str.strip()
df1.loc[:,'Road_traffic_density']=df1.loc[:,'Road_traffic_density'].str.strip()
df1.loc[:,'Type_of_vehicle']=df1.loc[:,'Type_of_vehicle'].str.strip()
df1.loc[:,'Festival']=df1.loc[:,'Festival'].str.strip()
df1.loc[:,'City']=df1.loc[:,'City'].str.strip()

In [5]:
df1.iloc[0,:]

ID                                          0x4607
Delivery_person_ID                  INDORES13DEL02
Delivery_person_Age                             37
Delivery_person_Ratings                        4.9
Restaurant_latitude                      22.745049
Restaurant_longitude                     75.892471
Delivery_location_latitude               22.765049
Delivery_location_longitude              75.912471
Order_Date                     2022-03-19 00:00:00
Time_Orderd                               11:30:00
Time_Order_picked                         11:45:00
Weatherconditions                 conditions Sunny
Road_traffic_density                          High
Vehicle_condition                                2
Type_of_order                                Snack
Type_of_vehicle                         motorcycle
multiple_deliveries                              0
Festival                                        No
City                                         Urban
Time_taken(min)                

## Grouping by


In [6]:
#nr of deliveres per city

cols = ['Delivery_person_Age','City','Type_of_vehicle' ]
#aux = ['ID', 'City']
#df1.loc[:,aux].groupby(['City']).size()
df1.loc[:,cols].groupby(['City', 'Type_of_vehicle']).min().reset_index()

,City,Type_of_vehicle,Delivery_person_Age
0,Metropolitian,bicycle,15
1,Metropolitian,electric_scooter,15
2,Metropolitian,motorcycle,15
3,Metropolitian,scooter,15
4,NaN,electric_scooter,20
5,NaN,motorcycle,20
6,NaN,scooter,20
7,Semi-Urban,electric_scooter,34
8,Semi-Urban,motorcycle,20
9,Semi-Urban,scooter,30


In [24]:
# mode 2
df1.groupby(['City', 'Type_of_vehicle'])[['Delivery_person_Age']].min()

Delivery_person_Age
City          Type_of_vehicle                      
Metropolitian bicycle                            15
              electric_scooter                   15
              motorcycle                         15
              scooter                            15
NaN           electric_scooter                   20
              motorcycle                         20
              scooter                            20
Semi-Urban    electric_scooter                   34
              motorcycle                         20
              scooter                            30
Urban         electric_scooter                   15
              motorcycle                         15
              scooter                            15

# Business Questions

In [7]:
#1. Qual o número total de entregadores cadastrados na base de dados?
df1.shape[0]

42805

In [8]:
#2. Qual o número total de entregadores únicos cadastrados na base de dados?
len(df1['ID'].unique())

42805

In [9]:
#2. Qual o número total de cidades unicas cadastrados na base de dados?
len(df1['City'].unique())

4

In [10]:
#2. Qual a média de idade dos entregadores por cidade?
cols = ['Delivery_person_Age', 'City']
df1.loc[:, cols].groupby(['City']).mean().reset_index()

,City,Delivery_person_Age
0,Metropolitian,29.792763
1,NaN,28.810860
2,Semi-Urban,32.565789
3,Urban,28.908480


In [22]:
df1.groupby(['City'])['Delivery_person_Age'].mean()

City
Metropolitian    29.792763
NaN              28.810860
Semi-Urban       32.565789
Urban            28.908480
Name: Delivery_person_Age, dtype: float64

In [11]:
# 3. Qual a mediana de idade dos entregadores por cidade?
cols = ['Delivery_person_Age', 'City']
df1.loc[:, cols].groupby(['City']).median().reset_index()


# Using Pivot Table
# median_df = df1.pivot_table(values=cols, index='City', aggfunc='median')
# median_df = median_df.reset_index()

,City,Delivery_person_Age
0,Metropolitian,30.0
1,NaN,28.0
2,Semi-Urban,32.0
3,Urban,28.0


In [12]:
# 4. Qual o número total de entregas feitas por densidade de tráfego?

cols = ['ID', 'Road_traffic_density']
df1.loc[:, cols].groupby(['Road_traffic_density']).size()

Road_traffic_density
High       4212
Jam       13480
Low       14605
Medium    10417
NaN          91
dtype: int64

In [13]:
# 5. Qual a média das avaliações das entregas feitas por densidade de tráfego?
cols = ['Delivery_person_Ratings', 'Road_traffic_density']
df1.loc[:, cols].groupby(['Road_traffic_density']).mean().reset_index()

,Road_traffic_density,Delivery_person_Ratings
0,High,4.653492
1,Jam,4.595395
2,Low,4.646780
3,Medium,4.660627
4,NaN,3.912088


In [14]:
# 6. Qual a quantidade de entregas feitas por cidade?
df1.loc[:, ['ID', 'City']].groupby(['City']).size()

City
Metropolitian    32031
NaN               1105
Semi-Urban         152
Urban             9517
dtype: int64

In [15]:
# # 7.Qual a média, mediana e desvio padrão das entregas feitas por cidade e por tipo de veículo?

# cols = ['ID', 'City', 'Type_of_vehicle']

# df1.loc[:, cols].groupby(['City', "Type_of_vehicle"]).agg({'City':['mean', 'median'],
#                                                            'Type_of_vehicle':['mean','median']})